In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
from autots import AutoTS
import plotly.io as pio

In [ ]:
pio.renderers.default = "notebook"
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False,
                      auto_adjust=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

In [ ]:
print(data.shape)
data.columns = [col[0] if col[1]=="" else col[0] for col in data.columns]
print(data.columns)

In [ ]:
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [ ]:
data.to_csv("data/btc_data.csv", index=False)

In [ ]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

In [ ]:
data = data[['Date', 'Close']]
data['Date'] = pd.to_datetime(data['Date'])
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data = data.dropna(subset=['Close']) 

In [ ]:
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple', model_list="fast", max_generations=5 )
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)